# Loan Default Prediction
## Team Members:
* Harish Puvvada
* Vamsi Mohan Ramineedi

In [1]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import HTML
pd.set_option("display.max_columns",75)

In [2]:
#df2007_11 = pd.read_csv(os.getenv('FDS')+'LoanStats_2007_to_2011.csv',low_memory=False,skiprows=1)
df2012_13 = pd.read_csv(os.getenv('FDS')+'LoanStats_2012_to_2013.csv',low_memory=False,skiprows=1)
df2014 = pd.read_csv(os.getenv('FDS')+'LoanStats_2014.csv',low_memory=False,skiprows=1)

In [3]:
dataset = pd.concat([df2012_13, df2014]) #merging 2007 to 2014 datasets
dataset = dataset.iloc[:,2:111] #removing empty columns
empty_cols = [i for i in range(45,72)] #more empty columns
dataset = dataset.drop(dataset.columns[empty_cols],axis=1)
data_with_loanstatus_sliced = dataset[(dataset['loan_status']=="Fully Paid") | (dataset['loan_status']=="Charged Off")]
di = {"Fully Paid":0, "Charged Off":1} #converting target variable to boolean
Dataset_withBoolTarget= data_with_loanstatus_sliced.replace({"loan_status": di})

In [4]:
#print(Dataset_withBoolTarget.shape)
dataset=Dataset_withBoolTarget.dropna(thresh = 340000,axis=1) #340000 is minimum number of non-NA values
#print(x.isnull().sum()) #- to check how many null values in all the columns
print(dataset.shape)
#dataset['loan_status'].value_counts()

(376233, 74)


In [5]:
del_col_names = ["delinq_2yrs",  "last_pymnt_d", "chargeoff_within_12_mths","delinq_amnt","emp_title", "term", "emp_title", "pymnt_plan","purpose","title", "zip_code", "verification_status", "dti","earliest_cr_line", "initial_list_status", "out_prncp",
"pymnt_plan", "num_tl_90g_dpd_24m", "num_tl_30dpd", "num_tl_120dpd_2m", "num_accts_ever_120_pd", "delinq_amnt", 
"chargeoff_within_12_mths", "total_rec_late_fee", "out_prncp_inv", "issue_d"]
dataset = dataset.drop(labels = del_col_names, axis = 1) 
dataset.shape


(376233, 52)

In [6]:
features = ['funded_amnt','emp_length','annual_inc','home_ownership','addr_state','tax_liens','grade',
            "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl",
            "mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy",
            "avg_cur_bal","acc_open_past_24mths",'loan_status']
Final_data = dataset[features] #19 features with target var

Final_data.shape

(376233, 20)

In [7]:
Final_data.head()

,funded_amnt,emp_length,annual_inc,home_ownership,addr_state,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
0,27050.0,10+ years,55000.0,OWN,MI,0.0,B,6074.19,4.0,0.0,10.99%,14.0,4.0,16.0,326.0,53.9,16473.0,9570.0,3.0,0
1,12000.0,4 years,60000.0,RENT,NC,0.0,B,3775.55,0.0,0.0,10.99%,15.0,7.0,1.0,48.0,15.9,15216.0,476.0,8.0,0
2,4800.0,2 years,39600.0,MORTGAGE,TX,0.0,B,3900.48,0.0,0.0,10.99%,3.0,2.0,25.0,220.0,16.1,21564.0,1379.0,0.0,0
3,12000.0,10+ years,130000.0,MORTGAGE,CO,0.0,B,398.28,3.0,0.0,11.99%,9.0,5.0,4.0,193.0,93.0,3567.0,36362.0,4.0,0
4,7550.0,3 years,28000.0,RENT,CA,0.0,C,529.67,0.0,0.0,16.24%,4.0,4.0,17.0,38.0,96.0,160.0,1440.0,1.0,0


features = []

mandatory  - "last_pymnt_amnt", "mort_acc", "pub_rec", "int_rate", "open_acc","num_actv_rev_tl","mo_sin_rcnt_rev_tl_op","mo_sin_old_rev_tl_op","bc_util","bc_open_to_buy","avg_cur_bal","acc_open_past_24mths"

doubtful - acc_now_delinq, pct_tl_nvr_dlq, num_sats, sub_grade

In [8]:
Final_data["funded_amnt"][2]

2     4800.0
2    21425.0
Name: funded_amnt, dtype: float64

In [9]:
Final_data.reset_index(drop=True, inplace = True)

In [10]:
Final_data["funded_amnt"][2]

4800.0

In [12]:
Final_data["int_rate"] = Final_data["int_rate"].apply(lambda x:float(x[:-1]) )

C:\Users\ramin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
Final_data["int_rate"].corr(Final_data["loan_status"])

0.24126296486971305

In [14]:
Final_data.head()

,funded_amnt,emp_length,annual_inc,home_ownership,addr_state,tax_liens,grade,last_pymnt_amnt,mort_acc,pub_rec,int_rate,open_acc,num_actv_rev_tl,mo_sin_rcnt_rev_tl_op,mo_sin_old_rev_tl_op,bc_util,bc_open_to_buy,avg_cur_bal,acc_open_past_24mths,loan_status
0,27050.0,10+ years,55000.0,OWN,MI,0.0,B,6074.19,4.0,0.0,10.99,14.0,4.0,16.0,326.0,53.9,16473.0,9570.0,3.0,0
1,12000.0,4 years,60000.0,RENT,NC,0.0,B,3775.55,0.0,0.0,10.99,15.0,7.0,1.0,48.0,15.9,15216.0,476.0,8.0,0
2,4800.0,2 years,39600.0,MORTGAGE,TX,0.0,B,3900.48,0.0,0.0,10.99,3.0,2.0,25.0,220.0,16.1,21564.0,1379.0,0.0,0
3,12000.0,10+ years,130000.0,MORTGAGE,CO,0.0,B,398.28,3.0,0.0,11.99,9.0,5.0,4.0,193.0,93.0,3567.0,36362.0,4.0,0
4,7550.0,3 years,28000.0,RENT,CA,0.0,C,529.67,0.0,0.0,16.24,4.0,4.0,17.0,38.0,96.0,160.0,1440.0,1.0,0
